In [1]:
# Comment out these lines
import sys
sys.path.insert(0, 'C:\\Users\\masch\\QuantumComputing\\QCompMAS\\pgmpy')

# Imports
import cmath
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.inference import BeliefPropagation

p = 0.64

bitFlip = BayesianNetwork([('q0m0', 'q0m1'), ('rv', 'q0m1'), ('q0m1', 'q0m2')])
cpd_q0m0 = TabularCPD(variable='q0m0', variable_card=2, values=[[1],[0]])
cpd_rv = TabularCPD(variable='rv', variable_card=2, values=[[np.sqrt(1-p)],[np.sqrt(p)]])
cpd_q0m1 = TabularCPD(variable='q0m1', variable_card=2, values=[[1,0,0,1],[0,1,1,0]], evidence=['q0m0', 'rv'], evidence_card = [2,2])
cpd_q0m2 = TabularCPD(variable='q0m2', variable_card=2, values=[[1/np.sqrt(2),1/np.sqrt(2)],[1/np.sqrt(2),-1/np.sqrt(2)]], evidence=['q0m1'], evidence_card=[2])
bitFlip.add_cpds(cpd_q0m0, cpd_rv, cpd_q0m1, cpd_q0m2)
BF_infer = VariableElimination(bitFlip)
bf = BF_infer.query(['rv', 'q0m2'])
print(bf)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+-------+---------+-----------------+
| rv    | q0m2    |    phi(rv,q0m2) |
+=======+=========+=================+
| rv(0) | q0m2(0) |  0.4243+0.0000j |
+-------+---------+-----------------+
| rv(0) | q0m2(1) |  0.4243+0.0000j |
+-------+---------+-----------------+
| rv(1) | q0m2(0) |  0.5657+0.0000j |
+-------+---------+-----------------+
| rv(1) | q0m2(1) | -0.5657+0.0000j |
+-------+---------+-----------------+


C:\Users\masch\QuantumComputing\QCompMAS\pgmpy\pgmpy\extern\tabulate.py:282: ComplexWarning: Casting complex values to real discards the imaginary part
  n = conv(string)


In [2]:
def cpd_2_dm(obj,rvs,var):
    numQubits = len(var)
    numRVs = len(rvs)
    varOrder = obj.variables
    numVars = len(varOrder)
    qubitOrdering = []
    rvsOrdering = []
    
    for i in range(numQubits):
        v = var[i]
        j = 0
        while(j < numVars and v != varOrder[j]):
            j += 1
        qubitOrdering.append(2**(numVars - j - 1))
        
    for i in range(numRVs):
        v = rvs[i]
        j = 0
        while(j < numVars and v != varOrder[j]):
            j += 1
        rvsOrdering.append(2**(numVars - j - 1))

    vals = (obj.values).flatten()
    dm = np.zeros((2**numQubits,2**numQubits),dtype="complex_")
    numEvents = 2**numRVs
    numPermutations = 2**numQubits
    
    for i in range(numEvents):
        val1 = 0
        for j in range(numRVs):
            val1 += ((i//(2**j))%2)*rvsOrdering[numRVs - j - 1]
        arr1 = np.zeros((numPermutations,1),dtype="complex_")
        arr2 = np.zeros((1,numPermutations),dtype="complex_")
        for j in range(numPermutations):
            val2 = val1
            for k in range(numQubits):
                val2 += ((j//(2**k))%2)*qubitOrdering[numQubits - k - 1]
            arr1[j][0] = vals[val2]
            arr2[0][j] = np.conj(vals[val2])
        dm += np.matmul(arr1,arr2)
        
    return dm

In [3]:
X = cpd_2_dm(bf,['rv'],['q0m2'])
print(X)

[[ 0.5 +0.j -0.14+0.j]
 [-0.14+0.j  0.5 +0.j]]


In [4]:
p2 = 0.64

bitFlip2 = BayesianNetwork([('q0m0', 'q0m1'), ('q1m0', 'q1m1'), ('rv', 'q1m1')])
cpd_q0m0 = TabularCPD(variable='q0m0', variable_card=2, values=[[1],[0]])
cpd_q1m0 = TabularCPD(variable='q1m0', variable_card=2, values=[[1],[0]])
cpd_rv = TabularCPD(variable='rv', variable_card=2, values=[[np.sqrt(1-p2)],[np.sqrt(p2)]])
cpd_q0m1 = TabularCPD(variable='q0m1', variable_card=2, values=[[1/np.sqrt(2),1/np.sqrt(2)],[1/np.sqrt(2),-1/np.sqrt(2)]], evidence=['q0m0'], evidence_card = [2])
cpd_q1m1 = TabularCPD(variable='q1m1', variable_card=2, values=[[1,0,0,1],[0,1,1,0]], evidence=['q1m0', 'rv'], evidence_card=[2,2])
bitFlip2.add_cpds(cpd_q0m0, cpd_q1m0, cpd_rv, cpd_q0m1, cpd_q1m1)
BF2_infer = VariableElimination(bitFlip2)
bf2 = BF2_infer.query(['rv', 'q0m1', 'q1m1'])
print(bf2)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+---------+---------+-------+---------------------+
| q0m1    | q1m1    | rv    |   phi(q0m1,q1m1,rv) |
+=========+=========+=======+=====================+
| q0m1(0) | q1m1(0) | rv(0) |      0.4243+0.0000j |
+---------+---------+-------+---------------------+
| q0m1(0) | q1m1(0) | rv(1) |      0.0000+0.0000j |
+---------+---------+-------+---------------------+
| q0m1(0) | q1m1(1) | rv(0) |      0.0000+0.0000j |
+---------+---------+-------+---------------------+
| q0m1(0) | q1m1(1) | rv(1) |      0.5657+0.0000j |
+---------+---------+-------+---------------------+
| q0m1(1) | q1m1(0) | rv(0) |      0.4243+0.0000j |
+---------+---------+-------+---------------------+
| q0m1(1) | q1m1(0) | rv(1) |      0.0000+0.0000j |
+---------+---------+-------+---------------------+
| q0m1(1) | q1m1(1) | rv(0) |      0.0000+0.0000j |
+---------+---------+-------+---------------------+
| q0m1(1) | q1m1(1) | rv(1) |      0.5657+0.0000j |
+---------+---------+-------+---------------------+


In [5]:
X = cpd_2_dm(bf2,['rv'],['q1m1', 'q0m1']).round(4)
print(X)

[[0.18+0.j 0.18+0.j 0.  +0.j 0.  +0.j]
 [0.18+0.j 0.18+0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.32+0.j 0.32+0.j]
 [0.  +0.j 0.  +0.j 0.32+0.j 0.32+0.j]]


In [6]:
from qiskit import QuantumCircuit
import qiskit.quantum_info as qi

circ1 = QuantumCircuit(2)
circ1.h(0)

circ2 = QuantumCircuit(2)
circ2.h(0)
circ2.x(1)

dm1 = (qi.DensityMatrix.from_instruction(circ1)).__array__()
dm2 = (qi.DensityMatrix.from_instruction(circ2)).__array__()

Y = ((1-p2)*dm1 + p2*dm2).round(4)
print(Y)

[[0.18+0.j 0.18+0.j 0.  +0.j 0.  +0.j]
 [0.18+0.j 0.18+0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.32+0.j 0.32+0.j]
 [0.  +0.j 0.  +0.j 0.32+0.j 0.32+0.j]]


In [7]:
X == Y

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])